Running this project require the following imports 

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import seaborn as sns 
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import sklearn.preprocessing as prep
from sklearn.datasets import make_blobs
from plotnine import *   
# StandardScaler is a function to normalize the data 
# You may also check MinMaxScaler and MaxAbsScaler 
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors
from sklearn.cluster import KMeans
from sklearn.cluster import AgglomerativeClustering
from sklearn.cluster import DBSCAN
from sklearn.mixture import GaussianMixture
from sklearn.metrics import silhouette_score
from sklearn.decomposition import PCA
from scipy.cluster.hierarchy import dendrogram
from scipy.cluster.hierarchy import linkage as la
%matplotlib inline

In [ ]:
def plot_clusters_PCA(data, kmeans=[], n_clusters=0):
    distortion=None
    if n_clusters==0:
        plt.scatter(data[:, 0], data[:, 1], c='b',alpha=0.5,s=20)
    else:
        # Perform clustering and plotting
        # Perform k-means clustering on the data
        labels = kmeans.labels_
        centroids = kmeans.cluster_centers_
        distortion= kmeans.inertia_
        # Perform PCA to reduce the data to 2 dimensions
        pca = PCA(n_components=2, random_state=0).fit(data)
        data_2d = pca.transform(data)
        centroids_2d = pca.transform(centroids)

        # Plot the data points and centroids in 2D
        plt.scatter(data_2d[:, 0], data_2d[:, 1], c=labels, cmap='rainbow',alpha=0.5,s=20)
        plt.scatter(centroids_2d[:, 0], centroids_2d[:, 1], marker='x', color='black')
        plt.xlabel('PC1')
        plt.ylabel('PC2')
        plt.title(f'Clusters of PCA data for k={n_clusters}')
        plt.show()
    return distortion

In [ ]:
def plot_clusters_nD(data, kmeans=[], n_clusters=0):
    distortion = None
    n_features = data.shape[1] # get the number of features
    n_rows = n_cols = int(np.ceil(np.sqrt(n_features))) # get the number of rows and columns for subplots
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(10, 10)) # create a figure with subplots
    fig.suptitle('Clusters of nD data') # set the figure title
    for i in range(n_features): # loop over the features
        row = i // n_cols # get the row index
        col = i % n_cols # get the column index
        ax = axes[row, col] # get the subplot axis
        if n_clusters == 0: # if no clustering is performed
            ax.scatter(data[:, i], data[:, (i+1) % n_features], c='b', alpha=0.5, s=20) # plot the data points
        else: # if clustering is performed
            # Perform k-means clustering on the data
            labels = kmeans.labels_
            centroids = kmeans.cluster_centers_
            distortion= kmeans.inertia_
            # Plot the data points and centroids in 2D
            ax.scatter(data[:, i], data[:, (i+1) % n_features], c=labels, cmap='rainbow', alpha=0.5, s=20) # plot the data points with cluster colors
            ax.scatter(centroids[:, i], centroids[:, (i+1) % n_features], marker='x', color='black') # plot the centroids with black crosses
        ax.set_xlabel('feature' + str(i+1)) # set the x label
        ax.set_ylabel('feature' + str((i+1) % n_features + 1)) # set the y label
    plt.tight_layout() # adjust the layout
    plt.show() # show the plot
    return distortion # return the distortion


In [ ]:
def display_clusters(data, algo=[]):
    n_features = data.shape[1] # get the number of features
    n_rows = n_cols = int(np.ceil(np.sqrt(n_features))) # get the number of rows and columns for subplots
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(10, 10)) # create a figure with subplots
    fig.suptitle('Clusters of nD data') # set the figure title
    for i in range(n_features): # loop over the features
        row = i // n_cols # get the row index
        col = i % n_cols # get the column index
        ax = axes[row, col] # get the subplot axis
        # Perform k-means clustering on the data
        labels = algo.labels_
        # Plot the data points and centroids in 2D
        ax.scatter(data[:, i], data[:, (i+1) % n_features], c=labels, cmap='rainbow', alpha=0.5, s=20) # plot the data points with cluster colors
        ax.set_xlabel('feature' + str(i+1)) # set the x label
        ax.set_ylabel('feature' + str((i+1) % n_features + 1)) # set the y label
    plt.tight_layout() # adjust the layout
    plt.show() # show the plot

### Gaussian Mixture
* Use GaussianMixture function to cluster the above data 
* In GMM change the covariance_type and check the difference in the resulting proabability fit 
* Use a 2D contour plot to plot the resulting distribution (the components of the GMM) as well as the total Gaussian mixture 

## iris data set 
The iris data set is test data set that is part of the Sklearn module 
which contains 150 records each with 4 features. All the features are represented by real numbers 

The data represents three classes 


In [ ]:
from sklearn.datasets import load_iris
iris_data_raw = load_iris()
iris_data_raw.target[[10, 25, 50]]
#array([0, 0, 1])
list(iris_data_raw.target_names)
['setosa', 'versicolor', 'virginica']
iris_data=iris_data_raw['data']

In [ ]:
mean = np.mean(iris_data,axis=0)
std = np.std(iris_data,axis=0)
min_val = np.min(iris_data,axis=0)
max_val = np.max(iris_data,axis=0)
norm = np.linalg.norm(iris_data,axis=0)

standard_iris = (iris_data - mean) / std
minMax_iris = (iris_data - min_val) / (max_val - min_val)
l2_iris = iris_data / norm
max_abs_iris = iris_data / np.abs(max_val)

iris_list = [{"data":standard_iris,"desc":"standardized_data"},\
    {"data":minMax_iris,"desc":"min_max_data"},\
        {"data":l2_iris,"desc":"l2_data"},\
        {"data":max_abs_iris,"desc":"max_abs_data"}]

* Repeat all the above clustering approaches and steps on the above data 
* Normalize the data then repeat all the above steps 
* Compare between the different clustering approaches 

#k-means

In [ ]:
K_range = range(2,11)
for i in iris_list:
    dist_func_iris=[]
    sil_scores_iris=[]
    data = i["data"]
    desc = i["desc"]
    print(f"graphs for {desc}")
    for k in K_range:
        kmeans = KMeans(n_clusters=k,init='k-means++',random_state=42)
        kmeans.fit(iris_data)
        plot_clusters_PCA(iris_data,kmeans,k)
        dist_func_iris.append(plot_clusters_nD(iris_data,kmeans,k))
        preds=kmeans.fit_predict(iris_data)
        sil_scores_iris.append(silhouette_score(iris_data,preds))

    plt.figure()
    # Create a line plot of the distortion function versus the K values
    plt.plot(K_range, dist_func_iris, marker='o')
    # Set the x and y labels
    plt.xlabel("Number of clusters")
    plt.ylabel("Distortion function")
    plt.title("Distortion Function vs Number of Clusters")
    # Show the plot
    plt.show()
    plt.figure()
    # Create a line plot of the Silhouette scores versus the K values
    plt.plot(K_range, sil_scores_iris, marker='o')
    # Set the x and y labels
    plt.xlabel("Number of clusters")
    plt.ylabel("Silhouette scores")
    plt.title("Silhouette scores vs Number of Clusters")
    # Show the plot
    plt.show()



#hierarchical

In [ ]:
affinities = ['euclidean','cityblock', 'cosine']
linkages = ['average', 'single']


sil_scores_iris2 = [[],[],[],[]]
corres_params_iris2 = [[],[],[],[]]
counter = 0
for i in iris_list:
    data = i["data"]
    desc = i["desc"]
    print(f"graphs for {desc}")
    # loop over the combinations of parameters
    for affinity in affinities:
        for linkage in linkages:
            # plot the dendrogram
            fig = plt.figure(figsize=(10, 5))
            Z = la(iris_data, method=linkage, metric=affinity)
            plt.title('Dendrogram for AgglomerativeClustering')
            plt.xlabel('Sample Index')
            plt.ylabel('Distance')
            dendrogram(Z)
            plt.show()
            distance_threshold_max=Z[:,2].max()
            step=distance_threshold_max/8
            array = np.arange(step,distance_threshold_max+step,step)
            distance_thresholds = array.tolist()

            for distance_threshold in distance_thresholds:
                # create an instance of the AgglomerativeClustering class
                ac = AgglomerativeClustering(n_clusters=None,metric=affinity, linkage=linkage, distance_threshold=distance_threshold)
                
                # fit the data and get the labels
                ac.fit(iris_data)
                labels = ac.labels_
                
                # compute the silhouette score
                if distance_threshold ==0:
                    # use the default number of clusters (2)
                    score = silhouette_score(iris_data, labels, metric=affinity)
                else:
                    # use the number of clusters found by the algorithm
                    score = silhouette_score(iris_data, labels, metric=affinity)
                
                sil_scores_iris2[counter].append(score)
                corres_params_iris2[counter].append({'Affinity':affinity,'Linkage':linkage,'Threshold':distance_threshold})
                
                # Perform PCA to reduce the data to 2 dimensions
                pca = PCA(n_components=2, random_state=0).fit(iris_data)
                data_2d = pca.transform(iris_data)
                
                # plot the clusters
                fig2 = plt.figure(figsize=(10, 5))
                plt.title(f'Affinity: {affinity}, Linkage: {linkage}, Distance Threshold: {distance_threshold}, silhouette score: {score}')
                plt.xlabel(f'Number of Clusters: {len(np.unique(labels))}')
                plt.scatter(data_2d[:, 0], data_2d[:, 1], c=labels, cmap='rainbow',alpha=0.5,s=20)
                plt.show()
                display_clusters(iris_data,ac)
    counter+=1


#DBscan

In [ ]:
# define the range of parameters
eps_values = np.arange(0.4, 1.6, 0.1)
min_samples_values = np.arange(5, 21, 1)

# initialize an empty list to store the silhouette scores
sil_scores_iris3 = [[],[],[],[]]
corres_params_iris3  = [[],[],[],[]]
counter = 0
for i in iris_list:
    data = i["data"]
    desc = i["desc"]
    print(f"graphs for {desc}")
    # loop over the combinations of parameters
    for eps in eps_values:
        for min_samples in min_samples_values:
            # create an instance of the DBSCAN class
            dbscan = DBSCAN(eps=eps, min_samples=min_samples)
            
            # fit the data and get the labels
            dbscan.fit(iris_data)
            labels = dbscan.labels_
            
            # compute the silhouette score
            # ignore the noise points (labelled as -1) for the score calculation
            score = silhouette_score(iris_data, labels)
            
            # append the score to the list
            sil_scores_iris3.append(score)
            corres_params_iris3.append( {"eps":eps, "min_samples":min_samples} )
            # Perform PCA to reduce the data to 2 dimensions
            pca = PCA(n_components=2, random_state=0).fit(iris_data)
            data_2d = pca.transform(iris_data)
            # plot the clusters
            fig = plt.figure(figsize=(10, 5))
            plt.title(f'EPS: {eps}, Min_samples: {min_samples}, silhouette score: {score}')
            plt.xlabel(f'Number of Clusters: {len(np.unique(labels)) - 1}') # subtract 1 to exclude the noise cluster
            plt.scatter(data_2d[:, 0], data_2d[:, 1], c=labels, cmap='rainbow',alpha=0.5,s=20)
            plt.show()
            display_clusters(iris_data,dbscan)
    counter+=1

# GMM

In [ ]:
from matplotlib.colors import LogNorm
# Define a function to display the clusters
def display_cluster(X, y_pred, title):
    plt.scatter(X[:, 0], X[:, 1], c=y_pred, s=50, cmap='viridis')
    plt.title(title)
    plt.show()

def plot_gmm_contours(gmm, X, title):
    # Create a grid of points to evaluate the GMM
    x = np.linspace(X[:, 0].min(), X[:, 0].max(), 100)
    y = np.linspace(X[:, 1].min(), X[:, 1].max(), 100)
    xx, yy = np.meshgrid(x, y)
    X_grid = np.c_[xx.ravel(), yy.ravel()]

    # Compute the log probability density of each point under each component
    z = gmm.score_samples(X_grid)
    z = z.reshape(xx.shape)
    # Compute the log probability density of each point under the total mixture
    log_prob_total = np.exp(z)
    
    # Plot the contours of the components and the total mixture
    plt.contour(xx, yy, log_prob_total, norm=LogNorm(vmin=1.0, vmax=1000.0), levels=14, cmap='viridis')
    plt.contour(xx, yy, log_prob_total, levels=14, colors='k')
    display_cluster(X, gmm.predict(X), title)

In [ ]:
counter = 0
for X in iris_list:
    data = i["data"]
    desc = i["desc"]
    print(f"graphs for {desc}")
    # Apply PCA to reduce the dimensionality to 2
    pca = PCA(n_components=2)
    X_pca = pca.fit_transform(X['data'])
    # Define a list of covariance types to try
    cov_types = ['full','spherical', 'diag', 'tied']

    # Loop over the covariance types and fit a GMM for each one
    for cov_type in cov_types:
        gmm = GaussianMixture(n_components=3, covariance_type=cov_type, random_state=42)
        gmm.fit(X_pca)
        y_pred = gmm.predict(X_pca)
        # Display the clusters
        display_cluster(X_pca, y_pred, f'GMM with 3 components and {cov_type} covariance')
        plot_gmm_contours(gmm, X_pca, f'GMM with 3 components and {cov_type} covariance')

## Customer dataset
Repeat all the above on the customer data set 

## k-means

In [ ]:
data_points=pd.read_csv("Customer data.csv",index_col=0)
data_points=data_points.to_numpy()

In [ ]:
mean = np.mean(data_points,axis=0)
std = np.std(data_points,axis=0)
min_val = np.min(data_points,axis=0)
max_val = np.max(data_points,axis=0)
norm = np.linalg.norm(data_points,axis=0)

standard_data_points = (data_points - mean) / std
minMax_data_points = (data_points - min_val) / (max_val - min_val)
l2_data_points = data_points / norm
max_abs_data_points = data_points / np.abs(max_val)

data_points_list = [{"data":standard_data_points,"desc":"standardized_data"},\
    {"data":minMax_data_points,"desc":"min_max_data"},\
        {"data":l2_data_points,"desc":"l2_data"},\
        {"data":max_abs_data_points,"desc":"max_abs_data"}]


In [ ]:
K_range = range(2,11)
for i in data_points_list:
    dist_func_data=[]
    sil_scores_data=[]
    data = i["data"]
    desc = i["desc"]
    print(f"graphs for {desc}")
    for k in K_range:
        kmeans = KMeans(n_clusters=k,init='k-means++',random_state=42)
        kmeans.fit(data)
        plot_clusters_PCA(data,kmeans,k)
        dist_func_data.append(plot_clusters_nD(data,kmeans,k))
        preds=kmeans.fit_predict(data)
        sil_scores_data.append(silhouette_score(data,preds,metric='euclidean'))

    plt.figure()
    # Create a line plot of the distortion function versus the K values
    plt.plot(K_range, dist_func_data, marker='o')
    # Set the x and y labels
    plt.xlabel("Number of clusters")
    plt.ylabel("Distortion function")
    plt.title("Distortion Function vs Number of Clusters")
    # Show the plot
    plt.show()
    plt.figure()
    # Create a line plot of the Silhouette scores versus the K values
    plt.plot(K_range, sil_scores_data, marker='o')
    # Set the x and y labels
    plt.xlabel("Number of clusters")
    plt.ylabel("Silhouette scores")
    plt.title("Silhouette scores vs Number of Clusters")
    # Show the plot
    plt.show()



# hierarchical

In [ ]:
affinities = ['euclidean','cityblock', 'cosine']
linkages = ['average', 'single']
sil_scores_data2 = [[],[],[],[]]
corres_params_data2 = [[],[],[],[]]
counter = 0
for i in data_points_list:
    data = i["data"]
    desc = i["desc"]
    print(f"graphs for {desc}")
    # loop over the combinations of parameters
    for affinity in affinities:
        for linkage in linkages:
            # plot the dendrogram
            fig = plt.figure(figsize=(10, 5))
            Z = la(data, method=linkage, metric=affinity)
            plt.title('Dendrogram for AgglomerativeClustering')
            plt.xlabel('Sample Index')
            plt.ylabel('Distance')
            dendrogram(Z)
            plt.show()
            distance_threshold_max=Z[:,2].max()
            step=distance_threshold_max/8
            array = np.arange(step,distance_threshold_max+step,step)
            distance_thresholds = array.tolist()

            for distance_threshold in distance_thresholds:
                # create an instance of the AgglomerativeClustering class
                ac = AgglomerativeClustering(n_clusters=None,metric=affinity, linkage=linkage, distance_threshold=distance_threshold)
                
                # fit the data and get the labels
                ac.fit(data)
                labels = ac.labels_
                
                # compute the silhouette score
                if distance_threshold ==0:
                    # use the default number of clusters (2)
                    score = silhouette_score(data, labels, metric=affinity)
                else:
                    # use the number of clusters found by the algorithm
                    score = silhouette_score(data, labels, metric=affinity)
                
                sil_scores_data2[counter].append(score)
                corres_params_data2[counter].append({'Affinity':affinity,'Linkage':linkage,'Threshold':distance_threshold})
                
                # Perform PCA to reduce the data to 2 dimensions
                pca = PCA(n_components=2, random_state=0).fit(data)
                data_2d = pca.transform(data)
                
                # plot the clusters
                fig2 = plt.figure(figsize=(10, 5))
                plt.title(f'Affinity: {affinity}, Linkage: {linkage}, Distance Threshold: {distance_threshold}, silhouette score: {score}')
                plt.xlabel(f'Number of Clusters: {len(np.unique(labels))}')
                plt.ylabel(f"{desc}")
                plt.scatter(data_2d[:, 0], data_2d[:, 1], c=labels, cmap='rainbow',alpha=0.5,s=20)
                plt.show()
                display_clusters(data,ac)
    counter+=1


# dbscan

In [ ]:
# define the range of parameters
eps_values = np.arange(.6, 1.6, 0.1)
min_samples_values = np.arange(2, 13, 1)

# initialize an empty list to store the silhouette scores
sil_scores_data3 = [[],[],[],[]]
corres_params_data3  = [[],[],[],[]]
counter = 0
for i in data_points_list:
    if counter!=1:
        counter+=1
        continue
    data = i["data"]
    desc = i["desc"]
    print(f"graphs for {desc}")
    
    # loop over the combinations of parameters
    for eps in eps_values:
        for min_samples in min_samples_values:
            # create an instance of the DBSCAN class
            dbscan = DBSCAN(eps=eps, min_samples=min_samples)
            
            # fit the data and get the labels
            dbscan.fit(data)
            labels = dbscan.labels_
            
            # compute the silhouette score
            # ignore the noise points (labelled as -1) for the score calculation
            score = silhouette_score(data, labels)
            
            # append the score to the list
            sil_scores_data3[counter].append(score)
            corres_params_data3[counter].append({"eps":eps, "min_samples":min_samples})
            # Perform PCA to reduce the data to 2 dimensions
            pca = PCA(n_components=2, random_state=0).fit(data)
            data_2d = pca.transform(data)
            # plot the clusters
            fig = plt.figure(figsize=(10, 5))
            plt.title(f'EPS: {eps}, Min_samples: {min_samples}, silhouette score: {score}')
            plt.xlabel(f'Number of Clusters: {len(np.unique(labels)) - 1}') # subtract 1 to exclude the noise cluster
            plt.ylabel(f'{desc}')
            plt.scatter(data_2d[:, 0], data_2d[:, 1], c=labels, cmap='rainbow',alpha=0.5,s=20)
            plt.show()
            display_clusters(data,dbscan)
    counter+=1

# GMM

In [ ]:
counter = 0
for X in data_points_list:
    data = X["data"]
    desc = X["desc"]
    print(f"graphs for {desc}")
    # Apply PCA to reduce the dimensionality to 2
    pca = PCA(n_components=2)
    X_pca = pca.fit_transform(X['data'])
    
    # Define a list of covariance types to try
    cov_types = ['full','spherical', 'diag', 'tied']

    # Loop over the covariance types and fit a GMM for each one
    for cov_type in cov_types:
        gmm = GaussianMixture(n_components=5, covariance_type=cov_type, random_state=42)
        gmm.fit(X_pca)
        y_pred = gmm.predict(X_pca)
        # Display the clusters
        display_cluster(X_pca, y_pred, f'GMM with 5 components and {cov_type} covariance')
        plot_gmm_contours(gmm, X_pca, f'GMM with 5 components and {cov_type} covariance')
